In [119]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from sklearn.metrics import roc_auc_score

In [120]:
MAX_SEQUENCE_LENGTH= 30
MAX_VOCAB_SIZE =20000
EMBEDDING_DIM=100
VALIDATION_SPLIT =0.2
BATCH_SIZE =128
EPOCHS =10

In [121]:
word2vec = {}
with open (os.path.join("glove.6B.100d.txt"), encoding='utf-8') as f:
    for line in f:
        values=line.split()
        word=values[0]
        vec = np.asarray(values[1:], dtype="float32")
        word2vec[word] =vec    

Set up the Data

In [122]:
df = pd.read_csv("popular_data.csv")
df.head()

,Unnamed: 0,askreddit,worldnews,funny,gaming,news,movies,tifu,mildlyinteresting,explainlikeimfive,...,todayilearned,jokes,aww,videos,lifeprotips,twoxchromosomes,oldschoolcool,art,dataisbeautiful,amitheasshole
0,0,Chris Rock is performing a standup gig tonight...,Zelenskyy says 'horrors' of Russian invasion c...,Will Smith arrives at the Oscars after party:,"A player in Elden Ring named ""Let me solo her""...",Jackson confirmed as first Black female high c...,"Hello, I’m Nicolas Cage and welcome to Ask Me ...",TIFU by thinking my son was having gay sex whe...,This pet shop don’t sell bunnies at Easter,ELI5: How do “hostile takeovers” work? Is ther...,...,TIL that after Lance Armstrong was stripped of...,We should have a TV show where illegal immigra...,When your cat trusts you so much that she brin...,Jim Carrey on Will Smith assaulting Chris Rock...,LPT If you're planning on visiting San Francis...,40% of teen pregnancies in 15 y/o girls involv...,Willem Dafoe 1981,"Cheers!, Me, Digital, 2022",[OC] I wondered why girls put their Snapchat i...,AITA for getting mad my artist hid their initi...
1,1,"What is your best insult, WITHOUT using curse ...",Russian warship Moskva has sunk - state media ...,"My wife, using her violin as weaponized sarcasm.",One of my favourite Easter Eggs in gaming,DC Police Find 5 Fetuses in Home of Anti-Abort...,"Gilbert Gottfried, Comedian and ‘Aladdin’ Star...",TIFU when my mother caught me eating my girlfr...,My dad wrote Isaac Asimov a question when he w...,ELI5: Why does the economy require to keep gro...,...,"TIL dogs ""play sneeze"" when they're feeling ex...",A lesbian named Linda went to the beach. She u...,Camping with Golden Retrievers,Johnny Depp’s reaction to the court finally ge...,LPT If you feel tired and want to sleep with k...,U.S. Congresswoman Marjorie Taylor Greene defi...,My grandma & grandpa in 1955,"Sis, get out the way I want to see the rainbow...",[OC] Which media organizations are trusted mor...,AITA for embarrassing my FIL after I repeatedl...


In [123]:
train = pd.DataFrame()
train['title'] = []
for i in df.columns[1:]:
    train[i] = []

for label in df.columns[1:]:
    for sentence in df[label]:
        train = train.append({'title': sentence, label:1},ignore_index=True)
        train = train.fillna(0)

train.head()


,title,askreddit,worldnews,funny,gaming,news,movies,tifu,mildlyinteresting,explainlikeimfive,...,todayilearned,jokes,aww,videos,lifeprotips,twoxchromosomes,oldschoolcool,art,dataisbeautiful,amitheasshole
0,Chris Rock is performing a standup gig tonight...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"What is your best insult, WITHOUT using curse ...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Zelenskyy says 'horrors' of Russian invasion c...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Russian warship Moskva has sunk - state media ...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Will Smith arrives at the Oscars after party:,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [124]:
possible_labels = ['askreddit',
'worldnews',
'funny',
'gaming',
'news',
'movies',
'tifu',
'mildlyinteresting',
'explainlikeimfive',
'pics',
'todayilearned',
'jokes',
'aww',
'videos',
'lifeprotips',
'twoxchromosomes',
'oldschoolcool',
'art',
'dataisbeautiful',
'amitheasshole']

Preprocessing and Tokenizing

In [125]:
from keras.preprocessing.text import Tokenizer

sentences = train["title"].fillna("NO_TITLE_EMPTY").values
targets = train[possible_labels].values

tokenizer= Tokenizer(num_words=MAX_VOCAB_SIZE)
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)
#sequences
#tokenizer.word_index
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

Preprocessing the Data Text

In [126]:
print("max sequence length:", max(len(s) for s in sequences))
print("min sequence length:", min(len(s) for s in sequences))
s = sorted(len(s) for s in sequences)
print("median sequence length:", s[len(s) // 2])

print("max word index:", max(max(seq) for seq in sequences if len(seq) > 0))


# get word -> integer mapping
word2idx = tokenizer.word_index
print('Found %s unique tokens.' % len(word2idx))

max sequence length: 53
min sequence length: 3
median sequence length: 15
max word index: 462
Found 462 unique tokens.


In [127]:
num_words = min(MAX_VOCAB_SIZE, len(word2idx) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

for word, i in word2idx.items():
  if i < MAX_VOCAB_SIZE:
    embedding_vector = word2vec.get(word)
    if embedding_vector is not None:
      # words not found in embedding index will be all zeros.
      embedding_matrix[i] = embedding_vector

In [128]:
embedding_layer = Embedding(
    num_words,
    EMBEDDING_DIM,
    weights=[embedding_matrix],
    input_length=MAX_SEQUENCE_LENGTH,
    trainable=False
)

In [129]:
input_ = Input(shape=(MAX_SEQUENCE_LENGTH,)) #Dimensions

x=embedding_layer(input_)

x=Conv1D(128,2,activation="relu")(x)
x=MaxPooling1D(2)(x)

x=Conv1D(128,3,activation="relu")(x)
x=MaxPooling1D(3)(x)

x=Conv1D(128,3,activation="relu")(x)
x=GlobalMaxPooling1D()(x)

x=Dense(128,activation="relu")(x)

output=Dense(len(possible_labels), activation="sigmoid")(x)

In [130]:
model = Model(input_, output)
model.compile(
  loss='binary_crossentropy',
  optimizer='rmsprop',
  metrics=['accuracy']
)

## Train the Model

In [131]:
r = model.fit(
  data,
  targets,
  batch_size=BATCH_SIZE,
  epochs=EPOCHS,
  validation_split=VALIDATION_SPLIT
)

Epoch 1/10
1/1 [==============================] - 3s 3s/step - loss: 0.7055 - accuracy: 0.0625 - val_loss: 0.5906 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 58ms/step - loss: 0.5293 - accuracy: 0.0938 - val_loss: 0.4538 - val_accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 0s 55ms/step - loss: 0.3269 - accuracy: 0.0625 - val_loss: 0.3617 - val_accuracy: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 0s 56ms/step - loss: 0.2125 - accuracy: 0.0625 - val_loss: 0.3626 - val_accuracy: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 0s 53ms/step - loss: 0.1882 - accuracy: 0.2188 - val_loss: 0.3632 - val_accuracy: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 0s 101ms/step - loss: 0.1738 - accuracy: 0.4688 - val_loss: 0.3758 - val_accuracy: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 0s 98ms/step - loss: 0.1658 - accuracy: 0.3438 - val_loss: 0.3811 - val_accuracy: 0.0000e+00


In [144]:
test_text = "WOMAN WOMAN WOMAN WOMAN WOMAN"
test_seq = tokenizer.texts_to_sequences(test_text)
test_pad = pad_sequences(test_seq,maxlen=MAX_SEQUENCE_LENGTH)
subreddits=model.predict(test_pad)[0].round(1)

In [145]:
len(possible_labels)

20

In [146]:
for i in range(0,20):
    print(subreddits[i] ,possible_labels[i])

0.5 askreddit
0.5 worldnews
0.5 funny
0.5 gaming
0.5 news
0.5 movies
0.5 tifu
0.5 mildlyinteresting
0.5 explainlikeimfive
0.5 pics
0.5 todayilearned
0.5 jokes
0.5 aww
0.5 videos
0.5 lifeprotips
0.5 twoxchromosomes
0.5 oldschoolcool
0.5 art
0.5 dataisbeautiful
0.5 amitheasshole
